In [1]:
!pip install transformers seqeval[gpu]

     |████████████████████████████████| 4.0 MB 25.0 MB/s 
     |████████████████████████████████| 43 kB 1.5 MB/s 
     |████████████████████████████████| 596 kB 60.1 MB/s 
     |████████████████████████████████| 6.6 MB 54.3 MB/s 
     |████████████████████████████████| 77 kB 8.7 MB/s 
     |████████████████████████████████| 895 kB 60.4 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=b79f3f7a235237a7cae537ce74d0a295c6c59b842688044fb7ad3c277525f33b
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertConfig, BertForTokenClassification, AutoTokenizer

In [3]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [4]:
sentence = []
labels = []
un_l = []
with open('train.txt') as f:
    lines = f.readlines()
    hello = 0
    # print(lines.split('\t')) 
    count = 0
    l = ""
    s = ""
    for val in lines:
      k = val.rstrip().split("\t")
      if len(s.split(" "))  == 50:
        #print(len(s.split(" ")))
        sentence.append(s)
        labels.append(l)
        l = ""
        s = ""
      #print(k)
      if len(k) >= 2:
        l = l + k[1] + ","
        s = s + k[0] + " "
        un_l.append(k[1])
      if len(k) < 2 and len(l) > 0:
        sentence.append(s)
        labels.append(l)
        l = ""
        s = ""
    f.close()

tags_ = un_l
ta = {}
un_lab = set(un_l)
unique_labels = []
for i in un_lab:
  unique_labels.append(i)
print(unique_labels)

for i in tags_:
  if i in ta:
    ta[i] += 1
  else:
    ta[i] = 0
print(ta)

data = {'sentence': sentence, 'labels': labels}  
df = pd.DataFrame(data) 
len(df)

df = df.loc[0:, ['sentence', 'labels']]

['Mineral', 'O', 'Element', 'Target']
{'O': 96134, 'Target': 712, 'Element': 1097, 'Mineral': 641}


In [5]:
print(df)

                                               sentence  \
0     . CENTIMETER TO DECIMETER SIZE SPHERICAL AND C...   
1     A.G. FairÃ©n8 , D. Oehler9 , O. Forni2 , V. Sa...   
2     , 7Brown , 8CAB , 9Jacobs LZ , 10MNHN , 11JPL/...   
3     observation has been southward-dipping sedimen...   
4     not been seen by previous landed missions . Th...   
...                                                 ...   
2034  complex also found wide variety of clays , car...   
2035  . Finally , there has been some evidence of re...   
2036  analysis of planetary analogs using multiple t...   
2037  potential processes that have taken place . If...   
2038  : -LSB- 1 -RSB- Grotzinger J. P. et al. -LRB- ...   

                                                 labels  
0     O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,...  
1     O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,...  
2     O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,...  
3     O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,...  
4

In [6]:
sentence = []
labels = []
un_l = []
with open('vali.txt') as f:
    lines = f.readlines()
    hello = 0
    # print(lines.split('\t')) 
    count = 0
    l = ""
    s = ""
    for val in lines:
      k = val.rstrip().split("\t")
      if len(s.split(" "))  == 50:
        #print(len(s.split(" ")))
        sentence.append(s)
        labels.append(l)
        l = ""
        s = ""
      #print(k)
      if len(k) >= 2:
        l = l + k[1] + ","
        s = s + k[0] + " "
        un_l.append(k[1])
      if len(k) < 2 and len(l) > 0:
        sentence.append(s)
        labels.append(l)
        l = ""
        s = ""
    f.close()

tags_ = un_l
ta = {}
un_lab = set(un_l)
unique_labels = []
for i in un_lab:
  unique_labels.append(i)
print(unique_labels)

for i in tags_:
  if i in ta:
    ta[i] += 1
  else:
    ta[i] = 0
print(ta)

validation_dataset = {'sentence': sentence, 'labels': labels}  
data_ = pd.DataFrame(validation_dataset) 
df_vali = df.loc[0:, ['sentence', 'labels']]

['Target', 'O', 'Mineral', 'Element']
{'O': 88990, 'Mineral': 620, 'Element': 914, 'Target': 364}


In [7]:
df_vali

,sentence,labels
0,. CENTIMETER TO DECIMETER SIZE SPHERICAL AND C...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
1,"A.G. FairÃ©n8 , D. Oehler9 , O. Forni2 , V. Sa...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
2,", 7Brown , 8CAB , 9Jacobs LZ , 10MNHN , 11JPL/...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
3,observation has been southward-dipping sedimen...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
4,not been seen by previous landed missions . Th...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
...,...,...
2034,"complex also found wide variety of clays , car...","O,O,O,O,O,O,O,O,Mineral,O,O,O,O,O,O,O,O,O,O,O,..."
2035,". Finally , there has been some evidence of re...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
2036,analysis of planetary analogs using multiple t...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
2037,potential processes that have taken place . If...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."


In [8]:
length = []
n = len(df)
for i in range(n):
  length.append(len(df.iloc[i].sentence))
print(max(length), min(length))

391 10


In [9]:
data = df
labels_to_ids = {k: v for v, k in enumerate(unique_labels)}
ids_to_labels = {v: k for v, k in enumerate(unique_labels)}
print(labels_to_ids)
print(ids_to_labels)

{'Target': 0, 'O': 1, 'Mineral': 2, 'Element': 3}
{0: 'Target', 1: 'O', 2: 'Mineral', 3: 'Element'}


In [10]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4
EPOCHS = 5
LEARNING_RATE = 2e-03
MAX_GRAD_NORM = 10
MODEL = 'bert-base-uncased'
tokenizer = BertTokenizerFast.from_pretrained(MODEL)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [11]:
class dataset(Dataset):
  def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

  def __getitem__(self, index):
        sentence = self.data.sentence[index]
        word_labels = self.data.labels[index].split(",") 
        encoding = self.tokenizer(sentence, return_offsets_mapping=True, padding='max_length', truncation=True, max_length=self.max_len)
        labels = []
        for label in  word_labels:
          if label == " " or label == "":
            pass
          else:
            labels.append(labels_to_ids[label])

        encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100

        i = 0
        for idx, mapping in enumerate(encoding["offset_mapping"]):
          if mapping[0] == 0 and mapping[1] != 0:
            encoded_labels[idx] = labels[i]
            i += 1

        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.as_tensor(encoded_labels)
        
        return item

  def __len__(self):
        return self.len

In [12]:
train_size = 0.80
train_dataset = data.sample(frac=train_size,random_state=200)
test_dataset = data.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)
vali_ = data_.sample(frac=0.999,random_state=200)
validation_dataset  = vali_.reset_index(drop=True)


training_set = dataset(train_dataset, tokenizer, MAX_LEN)
testing_set = dataset(test_dataset, tokenizer, MAX_LEN)
validation_set = dataset(validation_dataset, tokenizer, MAX_LEN)
#testing_set = dataset(train_dataset, tokenizer, MAX_LEN)

In [13]:
print(validation_set)

In [14]:
train_params = {'batch_size': TRAIN_BATCH_SIZE, 'shuffle': True, 'num_workers': 0}
test_params = {'batch_size': VALID_BATCH_SIZE,'shuffle': True,'num_workers': 0 }
vali_params = {'batch_size': VALID_BATCH_SIZE,'shuffle': True,'num_workers': 0 }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)
vali_loader = DataLoader(validation_set, **vali_params)

In [15]:
model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=len(labels_to_ids), output_attentions = False, 
    output_hidden_states = False, 
    #setting dropour probablities to 20%
    hidden_dropout_prob = 0.2,
    attention_probs_dropout_prob = 0.2 )
model.to(device)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.2, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [16]:
inputs = training_set[2]
input_ids = inputs["input_ids"].unsqueeze(0)
attention_mask = inputs["attention_mask"].unsqueeze(0)
labels = inputs["labels"].unsqueeze(0)

input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)

outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
initial_loss = outputs[0]
initial_loss

tensor(1.1765, device='cuda:0', grad_fn=<NllLossBackward0>)

In [17]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
  {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 
   'weight_decay_rate': 0.01
   },
  {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 
   'weight_decay_rate': 0.0}]

In [18]:
from transformers import  AdamW

In [19]:
optimizer = AdamW(optimizer_grouped_parameters, lr = 2e-5)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [20]:
# optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

In [21]:
train_accuracy = []
train_loss = []

In [22]:
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    tr_examples, tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    model.train()
    
    for idx, batch in enumerate(training_loader):
        # print(idx)
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        loss, tr_logits = model(input_ids=ids, attention_mask=mask, labels=labels, return_dict=False)
        tr_loss += loss.item()

        tr_steps += 1
        tr_examples += labels.size(0)
        
        if idx % 200==0:
            loss_step = tr_loss/tr_steps
            print("Training loss per 200 training steps: ",loss_step)
           
        flattened_targets = labels.view(-1) 
        active_logits = tr_logits.view(-1, model.num_labels) 
        flattened_predictions = torch.argmax(active_logits, axis=1) 
        

        active_accuracy = labels.view(-1) != -100 
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / tr_steps
    tr_accuracy = tr_accuracy / tr_steps
    print("Training loss epoch: ", epoch_loss)
    print("Training accuracy epoch: ", tr_accuracy)
    train_accuracy.append(epoch_loss)
    train_loss.append(tr_accuracy)

In [23]:
for epoch in range(10):
    print("Training epoch: ", epoch + 1)
    train(epoch)

Training epoch:  1
Training loss per 200 training steps:  1.4874355792999268
Training loss per 200 training steps:  0.13871549305607986
Training loss per 200 training steps:  0.12723399382177508
Training loss epoch:  0.12920426159489937
Training accuracy epoch:  0.9754901960784313
Training epoch:  2
Training loss per 200 training steps:  0.008434808813035488
Training loss per 200 training steps:  0.03790145564421233
Training loss per 200 training steps:  0.05248858772643245
Training loss epoch:  0.05190934844123309
Training accuracy epoch:  0.9852941176470589
Training epoch:  3
Training loss per 200 training steps:  0.04383720085024834
Training loss per 200 training steps:  0.026586008269168362
Training loss per 200 training steps:  0.03378686591988067
Training loss epoch:  0.0333964970826249
Training accuracy epoch:  0.9908088235294118
Training epoch:  4
Training loss per 200 training steps:  0.003633822314441204
Training loss per 200 training steps:  0.01095045462050313
Training loss

In [24]:
def valid(model, testing_loader):
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    eval_examples, eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            
            loss, eval_logits = model(input_ids=ids, attention_mask=mask, labels=labels, return_dict=False)
            
            eval_loss += loss.item()

            eval_steps += 1
            eval_examples += labels.size(0)
        
            if idx % 200==0:
                loss_step = eval_loss/eval_steps
                print("Validation loss per 200 evaluation steps: ",loss_step)
              

            flattened_targets = labels.view(-1) 
            active_logits = eval_logits.view(-1, model.num_labels) 
            flattened_predictions = torch.argmax(active_logits, axis=1) 
            
            active_accuracy = labels.view(-1) != -100 
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    labels = [ids_to_labels[id.item()] for id in eval_labels]
    predictions = [ids_to_labels[id.item()] for id in eval_preds]
    
    eval_loss = eval_loss / eval_steps
    eval_accuracy = eval_accuracy / eval_steps
    print("Validation Loss: ",eval_loss)
    print("Validation Accuracy: ",eval_accuracy)

    return labels, predictions

In [25]:
labels, predictions = valid(model, testing_loader)

Validation loss per 200 evaluation steps:  3.0755520128877833e-05
Validation Loss:  0.16141279414656381
Validation Accuracy:  0.9803921568627451


In [26]:
from seqeval.metrics import classification_report
from sklearn.metrics import confusion_matrix
# print(confusion_matrix(labels, predictions))
labels = [labels]
predictions = [predictions]

print(classification_report(labels, predictions))

              precision    recall  f1-score   support

       arget       1.00      0.67      0.80         3
      ineral       0.60      0.75      0.67         4
      lement       0.86      0.67      0.75         9

   micro avg       0.79      0.69      0.73        16
   macro avg       0.82      0.69      0.74        16
weighted avg       0.82      0.69      0.74        16



/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Element seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Mineral seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Target seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


In [27]:
O_ = labels[0].count('O')
M_ = labels[0].count("Mineral")
E_ = labels[0].count("Element")
T_ = labels[0].count("Target")

print(O_, M_, E_, T_)

392 4 9 3


In [28]:
print(labels)
print(predictions)

[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'Element', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'Mineral', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'Element', 'O', 'O', 'O', 'O', 'O', 'O', 'Element', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'Target', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'Mineral', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'Element', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'Element', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'Mineral', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'Element', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',

In [29]:
def valid(model, testing_loader):
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    eval_examples, eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            
            loss, eval_logits = model(input_ids=ids, attention_mask=mask, labels=labels, return_dict=False)
            
            eval_loss += loss.item()

            eval_steps += 1
            eval_examples += labels.size(0)
        
            if idx % 200==0:
                loss_step = eval_loss/eval_steps
                print("Validation loss per 200 evaluation steps: ",loss_step)
              

            flattened_targets = labels.view(-1) 
            active_logits = eval_logits.view(-1, model.num_labels) 
            flattened_predictions = torch.argmax(active_logits, axis=1) 
            
            active_accuracy = labels.view(-1) != -100 
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    labels = [ids_to_labels[id.item()] for id in eval_labels]
    predictions = [ids_to_labels[id.item()] for id in eval_preds]
    
    eval_loss = eval_loss / eval_steps
    eval_accuracy = eval_accuracy / eval_steps
    print("Validation Loss: ",eval_loss)
    print("Validation Accuracy: ",eval_accuracy)

    return labels, predictions

In [30]:
labels, predictions = valid(model, vali_loader)

Validation loss per 200 evaluation steps:  3.302042750874534e-05
Validation loss per 200 evaluation steps:  0.06468744660260788
Validation loss per 200 evaluation steps:  0.07528348852170176
Validation Loss:  0.06785987703576424
Validation Accuracy:  0.9877659574468085


In [31]:
labels = [labels]
predictions = [predictions]

print(classification_report(labels, predictions))

              precision    recall  f1-score   support

       arget       0.50      0.29      0.36         7
      ineral       0.62      0.83      0.71        12
      lement       0.68      0.76      0.72        17

   micro avg       0.64      0.69      0.67        36
   macro avg       0.60      0.63      0.60        36
weighted avg       0.63      0.69      0.65        36



/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Element seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Mineral seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Target seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


In [32]:
O_ = labels[0].count('O')
M_ = labels[0].count("Mineral")
E_ = labels[0].count("Element")
T_ = labels[0].count("Target")

print(O_, M_, E_, T_)

1843 12 17 7


In [65]:
print(labels)

[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'

In [41]:
print(predictions)

[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'Mineral', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'Element', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '

In [42]:
lables = labels[0]
predictions = predictions[0]

In [50]:
print(lables)
print(predictions)

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',